# Predizione del prezzo di un volo

**Progetto di Programmazione di Applicazioni Data Intensive - 2022/2023**  
Ingegneria e Scienze Informatiche  
Università di Bologna, Cesena

Silvia Furegato - 977475  
silvia.furegato@studio.unibo.it

---
## Obiettivo
L'obbiettivo di questo progetto è quello di predire il prezzo di un volo in base alle sue caratteristiche.

## Caricamento delle librerie

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

---
## Caricamento ed analisi dei dati
Cominciamo caricando un dataset sui voli ottenuto da [Kaggle](https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction) ed analizziamone la struttura.

In [4]:
import os.path as os

file = "Flight_Prices.csv";

if not os.exists(file):
    print("Missing dataset!");

dataset = pd.read_csv(file, index_col=0)

In [5]:
dataset.head(3)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956


In [6]:
dataset.tail(3)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585
300152,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,81585


In [7]:
dataset.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  object 
 1   flight            300153 non-null  object 
 2   source_city       300153 non-null  object 
 3   departure_time    300153 non-null  object 
 4   stops             300153 non-null  object 
 5   arrival_time      300153 non-null  object 
 6   destination_city  300153 non-null  object 
 7   class             300153 non-null  object 
 8   duration          300153 non-null  float64
 9   days_left         300153 non-null  int64  
 10  price             300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 155.1 MB


Vediamo che nel dataset sono presenti 300153 voli.  
Inoltre non sono presenti valori nulli.

### Significato delle colonne

Nel dataset sono presenti 11 colonne:  

- `Airline`: Il nome della compagnia aerea. Nel dataset sono presenti sei compagnie diverse.  
- `Flight`: Il codice di volo dell'aereo.  
- `Source City`: Città di partenza. Sono presenti seri città diverse.  
- `Departure Time`: Momento della giornata in cui l'aereo è partito. Anche per questo campo ci sono sei valori differenti.  
- `Stops`: Numero di scali. Ci sono tre valori possibili per questo campo.  
- `Arrival Time`: Momento della giornata in cui l'aereo giunge a destinazione.  
- `Destination City`: Città di arrivo.  
- `Class`: Classe. Ha due valori: Business and Economy.  
- `Duration`: Tempo di volo in ore.  
- `Days Left`: Quanti giorni prima del volo è stato acquistato il biglietto aereo.  
- `Price`: Prezzo del biglietto.  

La variabile che cercheremo di predire è `Price`.

Il codice del volo (`flight`) non è significativo, in quanto dipende dalla compagnia aerea e dalle città di partenza ed arrivo quindi lo elimineremo. Inoltre il campo `Arrival Time` può essere facilmente desunto da `Departure time` e `Duration`, quindi elimineremo anche quello.

In [8]:
dataset = dataset.drop(columns=["flight", "arrival_time"])
dataset.head(2)

,airline,source_city,departure_time,stops,destination_city,class,duration,days_left,price
0,SpiceJet,Delhi,Evening,zero,Mumbai,Economy,2.17,1,5953
1,SpiceJet,Delhi,Early_Morning,zero,Mumbai,Economy,2.33,1,5953


In [ ]:
#rinomina colonne se serve

Sono presenti molte colonne di tipo object, trasformiamole in tipo categorico.

In [11]:
categorical_fields = ["airline", "source_city", "departure_time", "stops", "destination_city", "class"];

for field in categorical_fields:
    dataset[field] = dataset[field].astype("category")

In [13]:
dataset.dtypes

airline             category
source_city         category
departure_time      category
stops               category
destination_city    category
class               category
duration             float64
days_left              int64
price                  int64
dtype: object

In [12]:
dataset.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300153 entries, 0 to 300152
Columns: 9 entries, airline to price
dtypes: category(6), float64(1), int64(2)
memory usage: 10.9 MB


Come possiamo vedere, l'utilizzo di memoria è diminuito drasticamente.  
Vediamo ora alcune statistiche sui campi numerici.

In [9]:
dataset.describe()

,duration,days_left,price
count,300153.000000,300153.000000,300153.000000
mean,12.221021,26.004751,20889.660523
std,7.191997,13.561004,22697.767366
min,0.830000,1.000000,1105.000000
25%,6.830000,15.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.170000,38.000000,42521.000000
max,49.830000,49.000000,123071.000000


In [18]:
dataset["airline"].unique().size

6

In [13]:
#conteggio records e num per ogni categoria?

In [14]:
#quali sono categoriche? continue?

In [15]:
#grafici

---
## Preparazione dei dati